In [3]:
#import
import scipy.io

In [9]:
#Data load
#EEG - Change.mat       C:\Users\USER\Desktop\이소현\EEG\B2X_2nd_Year_3rd_Pilot_Customizing\matlab
#ECG - ECG_feature_~    C:\Users\USER\Desktop\이소현\HRV\EDA\ECG_EDA
EEG_data_path = 'C:\\Users\\USER\\Desktop\\이소현\\EEG\\B2X_2nd_Year_3rd_Pilot_Customizing\\matlab\\Change.mat'
ECG_data_path_S0 = 'C:\\Users\\USER\\Desktop\\이소현\\HRV\\EDA\\ECG_EDA\\ECG_feature_S0.mat'
ECG_data_path_N020 = 'C:\\Users\\USER\\Desktop\\이소현\\HRV\\EDA\\ECG_EDA\\ECG_feature_N020.mat'

EEG_data = scipy.io.loadmat(EEG_data_path)